In [1]:
import scipy.optimize
import csv
import random
from sklearn import linear_model
from collections import defaultdict

def parseDataFromFile(fname):
    cursor = open(fname,'r')
    csvreader = csv.reader(cursor,delimiter=';')
    return csvreader
      
print "Reading data..."
data = parseDataFromFile("Assignment2/database.csv")
data = list(data)
print "done"


### Preprocessing data ###
crime_data = [l[0].split(",") for l in data[1:]]
race_type = defaultdict(int)
crime_train_data = crime_data[:300000]
random.shuffle(crime_train_data)

Reading data...


done


In [3]:
##### Predicting Killer Age #####

### data initialization ###
states_age = defaultdict(int) # store average age for each state
states_number = defaultdict(int) # store number of criminal in each state
global_avgAge = 0 # global average age for killer
counter = 0

features = []
age = []

for l in crime_train_data:
    if "Yes" in l[10] and (int(l[16]) < 100 and int(l[16]) > 0):
        states_age[l[5]] += int(l[16])
        states_number[l[5]] += 1
        global_avgAge += int(l[16])
        counter += 1
            
for s in states_age:
    states_age[s] = states_age[s] * 1.0 / states_number[s]
global_avgAge = global_avgAge * 1.0 / counter


### Data Training ###
for l in crime_train_data:
    if "Yes" in l[10] and (int(l[16]) < 100 and int(l[16]) > 0):
        weapon = ["Handgun" in l[20], "Knife" in l[20], "Blunt Object" in l[20], "Firearm" in l[20],
                  "Unknown" in l[20], "Shotgun" in l[20], "Rifle" in l[20], "Strangulation" in l[20],
                  "Fire" in l[20], "Suffocation" in l[20], "Gun" in l[20], "Drugs" in l[20],
                  "Drowning" in l[20], "Explosives" in l[20], "Poison" in l[20], "Fall" in l[20]]
        relationship = ["Stepmother" in l[19], "Employee" in l[19], "Employer" in l[19],
                        "Ex-Husband" in l[19], "Stepdaughter" in l[19], "Stepson" in l[19],
                        "Sister" in l[19], "Stepfather" in l[19], "friend" in l[19],
                        "Common-Law Husband" in l[19], "Ex-Wife" in l[19], "Common-Law Wife" in l[19],
                        "In-Law" in l[19], "Mother" in l[19], "Father" in l[19],
                        "Brother" in l[19], "Neighbor" in l[19],
                        "Daughter" in l[19], "Husband" in l[19], "Family" in l[19],
                        "Son" in l[19], "Friend" in l[19], "Wife" in l[19],
                        "Stranger" in l[19], "Acquaintance" in l[19]]
        killer_ethnicity = ["Unknown" in l[18], l[18] == "Not Hispanic", l[18] == "Hispanic"]
        victim_ethnicity = ["Unknown" in l[14], l[14] == "Not Hispanic", l[14] == "Hispanic"]
        victim_race = ["White" in l[13], "Black" in l[13], "Native American" in l[13], 
                       "Asian" in l[13], "Unknown" in l[13]]
        victim_sex = ["Male" in l[11], "Female" in l[11]]
        victim_age = [int(l[12])]
        killer_sex = ["Male" in l[15], "Female" in l[15]]
        killer_race = ["White" in l[17], "Black" in l[17], "Native American" in l[17], 
                       "Asian" in l[17], "Unknown" in l[17]]
        year = [int(l[6])]
        
        #bias = states_age[l[5]] - global_avgAge
        
        # Know nothing about the killer
        feature_arr = year + victim_ethnicity + victim_sex + victim_age + victim_race + killer_sex + killer_race + [1]
        # Train killer age
        age.append(int(l[16]))
        features.append(feature_arr)

In [5]:
#### Logistic Regression Training ####
train_features = features
train_age = age

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(train_features, train_age)
theta = clf.coef_
train_age_predict = clf.predict(train_features)

# Linear Regression
#theta,residuals,rank,s = numpy.linalg.lstsq(train_features, train_age)

errors = [abs(a - b) for (a,b) in zip(train_age_predict, train_age)]
MAE_train = sum(errors)*1.0 / len(train_age_predict)
print "Training: MAE for predicting killer's age = ", MAE_train

Training: MAE for predicting killer's age =  8.93518556736


In [6]:
##### Predicting Killer Age #####
features = []
age = []
age_bias = []

### Data Training ###
for l in crime_data[300000:350000]:
    if "Yes" in l[10] and (int(l[16]) > 0 and int(l[16]) < 100):
        weapon = ["Handgun" in l[20], "Knife" in l[20], "Blunt Object" in l[20], "Firearm" in l[20],
                  "Unknown" in l[20], "Shotgun" in l[20], "Rifle" in l[20], "Strangulation" in l[20],
                  "Fire" in l[20], "Suffocation" in l[20], "Gun" in l[20], "Drugs" in l[20],
                  "Drowning" in l[20], "Explosives" in l[20], "Poison" in l[20], "Fall" in l[20]]
        relationship = ["Stepmother" in l[19], "Employee" in l[19], "Employer" in l[19],
                        "Ex-Husband" in l[19], "Stepdaughter" in l[19], "Stepson" in l[19],
                        "Sister" in l[19], "Stepfather" in l[19], "friend" in l[19],
                        "Common-Law Husband" in l[19], "Ex-Wife" in l[19], "Common-Law Wife" in l[19],
                        "In-Law" in l[19], "Mother" in l[19], "Father" in l[19],
                        "Brother" in l[19], "Neighbor" in l[19],
                        "Daughter" in l[19], "Husband" in l[19], "Family" in l[19],
                        "Son" in l[19], "Friend" in l[19], "Wife" in l[19],
                        "Stranger" in l[19], "Acquaintance" in l[19]]
        killer_ethnicity = ["Unknown" in l[18], l[18] == "Not Hispanic", l[18] == "Hispanic"]
        victim_ethnicity = ["Unknown" in l[14], l[14] == "Not Hispanic", l[14] == "Hispanic"]
        victim_race = ["White" in l[13], "Black" in l[13], "Native American" in l[13], 
                       "Asian" in l[13], "Unknown" in l[13]]
        victim_sex = ["Male" in l[11], "Female" in l[11]]
        victim_age = [int(l[12])]
        killer_sex = ["Male" in l[15], "Female" in l[15]]
        killer_race = ["White" in l[17], "Black" in l[17], "Native American" in l[17], 
                       "Asian" in l[17], "Unknown" in l[17]]
        year = [int(l[6])]
        
        bias = states_age[l[5]] - global_avgAge
        
        # Know nothing about the killer
        feature_arr =  year + victim_ethnicity + victim_sex  + victim_age + victim_race + killer_sex + killer_race + [1]
        # Train killer age
        age.append(int(l[16]))
        features.append(feature_arr)
        age_bias.append(bias)

In [7]:
#### Logistic Regression Testing ###
test_features = features
test_age = age
test_age_predict = []

prediction = clf.predict(test_features)

for i in range(0,len(prediction)):
    if prediction[i] <= 18:
        test_age_predict.append(global_avgAge)
    elif prediction[i] >= 55:
        test_age_predict.append(global_avgAge)
        #test_age_predict[i] = global_avgAge + age_bias[i]*0.4
    else:
        test_age_predict.append(prediction[i] + age_bias[i]*0.5)
#test_age_predict = [x + y*0.4 for (x,y) in zip(prediction, age_bias)]
MAE_test = sum([abs(round(a) - b) for (a,b) in zip(test_age_predict, test_age)]) * 1.0 \
            / len(test_age_predict)
print "Testing: MAE for predicting killer's age = ", MAE_test

Testing: MAE for predicting killer's age =  8.52921334012


In [70]:
MAE_test = sum([abs(global_avgAge - b) for (a,b) in zip(test_age_predict, test_age)]) * 1.0 \
            / len(test_age_predict)
print "Testing: MAE for trivially predicting killer's age = ", MAE_test

Testing: MAE for trivially predicting killer's age =  9.98066221103
